<a href="https://colab.research.google.com/github/dolDolSee/TF2.0/blob/main/Regression_boston.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd 
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf


boston = load_boston()
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF['PRICE'] = boston.target

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])

In [ ]:
class Regression(tf.keras.Model):
    def __init__(self):
        super(Regression, self).__init__()
        self.regression_layer = tf.keras.layers.Dense(1,
                                                   activation = None,
                                                   #W값
                                                   kernel_initializer = "zeros",
                                                   #B값
                                                   bias_initializer="zeros")
    def call(self, x):
        #인자를 넣어서 softmax_layer연산 수행
        logits = self.regression_layer(x)
        #Dense를 지난 결과에 softmax 씌워서 반환
        return logits
#손실함수 정의
@tf.function
def mse_loss(y_pred, y):
    return tf.reduce_mean(tf.square(y_pred-y))

#Optimizer
optimizer = tf.optimizers.Adam(0.01)

#GD정의
@tf.function
def train_step(model, x, y):
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = mse_loss(y_pred, y)
    #GD가 계산되는 시점, model.trainable_variable은 자동으로 갱신이 요구되는 변수값을 할당해줌.
    gradients = tape.gradient(loss, model.trainable_variables)
    #Optimizer를 통해서 갱신이 이루어지는 시점
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))

Regression_model = Regression()

Regression_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01),loss='mse',metrics=['mse'])
Regression_model.fit(scaled_features, bostonDF['PRICE'].values, epochs=1000)


In [3]:
predicted = Regression_model.predict(scaled_features)
predicted[0]

array([28.958841], dtype=float32)